<a href="https://colab.research.google.com/github/alijawad480/Fake-News-Classifier-NLP/blob/main/Fake_News_Classifier_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('/content/WELFake_Dataset.csv')

In [3]:
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [4]:
print(list(df.columns))

['Unnamed: 0', 'title', 'text', 'label']


In [5]:
df.isnull().sum()

,0
Unnamed: 0,0
title,558
text,39
label,0


In [6]:
df.shape

(72134, 4)

In [7]:
df=df.head(50000)

In [8]:
df=df.dropna()

In [9]:
df['content'] = df['title'].astype(str) + " " + df['text'].astype(str)
X=df['content']
y=df['label']

In [10]:
import tensorflow as tf

In [11]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout,Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot

In [12]:
voc_size=5000

In [13]:
print(df['title'].iloc[0])

LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11By #BlackLivesMatter And #FYF911 Terrorists [VIDEO]


In [14]:
import nltk
import re
from nltk.corpus import stopwords

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
#df['content'].iloc[i]

In [18]:
stop_words = set(stopwords.words('english'))

In [19]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
corpus = []
for i in range(len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['content'].iloc[i])
    review = review.lower().split()
    review = [ps.stem(word) for word in review if word not in stop_words]
    corpus.append(' '.join(review))

In [20]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]

In [21]:
corpus[1]

'unbeliev obama attorney gener say charlott rioter peac protest home state north carolina video demonstr gather last night exercis constitut protect right peac protest order rais issu creat chang loretta lynch aka eric holder skirt'

In [22]:
onehot_repr[1]

[4430,
 3501,
 2796,
 626,
 1812,
 1914,
 2221,
 1101,
 537,
 970,
 1191,
 3363,
 2424,
 3320,
 3372,
 1916,
 1812,
 3062,
 1675,
 2228,
 2290,
 2170,
 1101,
 537,
 1954,
 2173,
 4590,
 2301,
 3370,
 1389,
 1649,
 1983,
 1345,
 1120,
 2304]

In [23]:
sent_length=50
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

In [24]:
print(embedded_docs)

[[3789  865 3320 ... 4703 4963 3996]
 [   0    0    0 ... 1345 1120 2304]
 [4461 2809 3789 ... 3679 1812 1446]
 ...
 [1191 1151 1675 ...  584 1954 3709]
 [2505 3724 4810 ... 2135  353 2577]
 [3780 3788 3153 ... 4408 2379  855]]


In [25]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 4430, 3501, 2796,  626, 1812, 1914, 2221,
       1101,  537,  970, 1191, 3363, 2424, 3320, 3372, 1916, 1812, 3062,
       1675, 2228, 2290, 2170, 1101,  537, 1954, 2173, 4590, 2301, 3370,
       1389, 1649, 1983, 1345, 1120, 2304], dtype=int32)

In [26]:
from tensorflow.keras import regularizers
embending_vectroze_feature=40
model=Sequential()
model.add(Embedding(input_dim=voc_size, output_dim=embending_vectroze_feature,
                    input_length=sent_length, mask_zero=True))
model.add(LSTM(28,return_sequences=False,kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(28,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1,activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [27]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000,
    decay_rate=0.9
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [28]:
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
model.build(input_shape=(None, 28))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 28, 40)         │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 28)             │         7,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 28)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 28)             │           812 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 28)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            29 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 208,569 (814.72 KB)

 Trainable params: 208,569 (814.72 KB)

 Non-trainable params: 0 (0.00 B)

In [29]:
import numpy as np
X_final = np.array(embedded_docs)
y_final=np.array(y)

In [30]:
import numpy as np
unique, counts = np.unique(y_final, return_counts=True)
print(dict(zip(unique, counts)))

{np.int64(0): np.int64(24246), np.int64(1): np.int64(25366)}


In [31]:
X_final.shape

(49612, 50)

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=42)

In [33]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5,batch_size=32)

Epoch 1/5
1241/1241 ━━━━━━━━━━━━━━━━━━━━ 46s 34ms/step - accuracy: 0.8337 - loss: 0.4079 - val_accuracy: 0.9109 - val_loss: 0.2325
Epoch 2/5
1241/1241 ━━━━━━━━━━━━━━━━━━━━ 80s 32ms/step - accuracy: 0.9332 - loss: 0.1984 - val_accuracy: 0.9130 - val_loss: 0.2280
Epoch 3/5
1241/1241 ━━━━━━━━━━━━━━━━━━━━ 40s 32ms/step - accuracy: 0.9495 - loss: 0.1552 - val_accuracy: 0.9169 - val_loss: 0.2266
Epoch 4/5
1241/1241 ━━━━━━━━━━━━━━━━━━━━ 40s 32ms/step - accuracy: 0.9596 - loss: 0.1304 - val_accuracy: 0.9158 - val_loss: 0.2492
Epoch 5/5
1241/1241 ━━━━━━━━━━━━━━━━━━━━ 43s 34ms/step - accuracy: 0.9679 - loss: 0.1075 - val_accuracy: 0.9148 - val_loss: 0.2912
